# $^{222}\text{Rn}$-$\alpha$ in $\text{Xe}$

Analysis plots for the $^{214}\text{Po}$-like population in $\alpha$ analysis.

In [2]:
import sys
sys.path.append('/data_extra2/ccortesp/NEXT-100/Xe_cmmssnng/')

import crudo
import fit_functions as ff
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import plotting_tools as pt
import tables as tb
from   scipy.optimize import curve_fit

%matplotlib inline
%load_ext autoreload
%autoreload 2

## Runs Information

In [3]:
# Select the type of scan in your analysis
# scan, scan_key = "EL", "EL_HV"
# scan, scan_key = "Cathode", "cath_HV"

# Select the runs of interest
runs_info = [
                # Drift field scan: Oct 2024
                # {"run_number": 14440, "cath_HV": 18e3, "EL_HV": 6e3, "duration": 42952},
                # {"run_number": 14456, "cath_HV": 19e3, "EL_HV": 6e3, "duration": 56420},
                # {"run_number": 14468, "cath_HV": 20e3, "EL_HV": 6e3, "duration": 55696},
                # {"run_number": 14484, "cath_HV": 21e3, "EL_HV": 6e3, "duration": 48240},
                # {"run_number": 14489, "cath_HV": 22e3, "EL_HV": 6e3, "duration": 42314},
                # {"run_number": 14498, "cath_HV": 23e3, "EL_HV": 6e3, "duration": 43161},
                # {"run_number": 14519, "cath_HV": 23e3, "EL_HV": 6e3, "duration": 43274},
                # {"run_number": 14522, "cath_HV": 23e3, "EL_HV": 6e3, "duration": 42801},
                # --- Gas recirculation: Nov 29 --- #
                # {"run_number": 14581, "cath_HV": 23e3, "EL_HV": 6e3, "duration": 76086},
                # {"run_number": 14582, "cath_HV": 23e3, "EL_HV": 6e3, "duration": 79541},
                # {"run_number": 14588, "cath_HV": 23e3, "EL_HV": 6e3, "duration": 55856},
                # EL voltage scan: Dic 2024
                # {"run_number": 14601, "cath_HV": 23e3, "EL_HV": 7e3, "duration": 77537},
                # {"run_number": 14608, "cath_HV": 23e3, "EL_HV": 8e3, "duration": 86414},
                # {"run_number": 14704, "cath_HV": 23e3, "EL_HV": 9e3, "duration": 52765},
    
                # # # # #  A L P H A   R U N S # # # # #
    
                # # Jan 2025
                # {"run_number": 14714, "duration": 64527, "OK": -1     , "LOST": -1     , "real_rate": -1    },
                # {"run_number": 14715, "duration": 84365, "OK": 2469062, "LOST": 2239303, "real_rate": 55.809},  # NO processed 
                # {"run_number": 14716, "duration": 17036, "OK": 495769 , "LOST": 451306 , "real_rate": 55.592},
                # {"run_number": 14720, "duration": 48518, "OK": -1     , "LOST": -1     , "real_rate": -1    },
                # {"run_number": 14733, "duration": 53881, "OK": 1587637, "LOST": 1429629, "real_rate": 55.998},
                # {"run_number": 14735, "duration": 84987, "OK": 2508569, "LOST": 2267982, "real_rate": 56.203},
                # {"run_number": 14737, "duration": 72705, "OK": 2153786, "LOST": 1960347, "real_rate": 56.586},
                {"run_number": 14739, "duration": 87138, "OK": 2576630, "LOST": 2334343, "real_rate": 56.358},
                # {"run_number": 14741, "duration": 87755, "OK": 2592615, "LOST": 2349583, "real_rate": 56.318},
                # {"run_number": 14743, "duration": 82332, "OK": 2437984, "LOST": 2220643, "real_rate": 56.583},
                # {"run_number": 14745, "duration": 60990, "OK": 1803415, "LOST": 1636317, "real_rate": 56.398},
                # # --- NO Cold Getter: Jan 27 --- #
                # {"run_number": 14753, "duration": 86308, "OK": 2188547, "LOST": 1532351, "real_rate": 43.111},
                # {"run_number": 14765, "duration": 55978, "OK": 1259832, "LOST": 735202 , "real_rate": 35.639},
                # {"run_number": 14776, "duration": 47397, "OK": 957081 , "LOST": 481810 , "real_rate": 30.358},  # Data removed
                # {"run_number": 14780, "duration": 88503, "OK": 1600797, "LOST": 682246 , "real_rate": 25.796},
                # # Feb 2025
                # {"run_number": 14782, "duration": 85654, "OK": 1382153, "LOST": 506234 , "real_rate": 22.046},
                # {"run_number": 14784, "duration": 57290, "OK": 842541 , "LOST": 273581 , "real_rate": 19.481},
                # {"run_number": 14789, "duration": 74148, "OK": 976360 , "LOST": 277504 , "real_rate": 16.910},
                # # --- Zero Suppression --- #
                # {"run_number": 14803, "duration": 71721, "OK": 933727 , "LOST": 104677 , "real_rate": 14.478},
                # {"run_number": 14804, "duration": 65116, "OK": 680552 , "LOST": 156995 , "real_rate": 12.862},
                # # --- NO Zero Suppression --- #
                # {"run_number": 14811, "duration": -1   , "OK": -1     , "LOST": -1     , "real_rate": -1    },
                # {"run_number": 14814, "duration": 6344 , "OK": 55679  , "LOST": 10155  , "real_rate": 10.377},
                # {"run_number": 14815, "duration": 86232, "OK": 717786 , "LOST": 121527 , "real_rate": 9.733 },
                # {"run_number": 14816, "duration": 86580, "OK": 659265 , "LOST": 101374 , "real_rate": 8.785 },
                # {"run_number": 14817, "duration": 49662, "OK": 352203 , "LOST": 50474  , "real_rate": 8.108 },
                # {"run_number": 14828, "duration": 53609, "OK": 300827 , "LOST": 100405 , "real_rate": 7.484 },
                # {"run_number": 14829, "duration": 73042, "OK": 387003 , "LOST": 121249 , "real_rate": 6.958 },
                # {"run_number": 14834, "duration": 5779 , "OK": 32864  , "LOST": 3887   , "real_rate": 6.359 },
                # {"run_number": 14835, "duration": 11713, "OK": 66515  , "LOST": 7358   , "real_rate": 6.306 },
                # {"run_number": 14837, "duration": 55751, "OK": 294175 , "LOST": 31034  , "real_rate": 5.833 },
                # {"run_number": 14838, "duration": 87854, "OK": 431766 , "LOST": 42219  , "real_rate": 5.395 },
                # {"run_number": 14839, "duration": 84881, "OK": 401004 , "LOST": 37747  , "real_rate": 5.169 },
                # {"run_number": 14840, "duration": 55363, "OK": -1     , "LOST": -1     , "real_rate": -1    },
            ]

### Selected Data

The idea is to load the Polike and NO-Polike dataframes for comparisons.

In [48]:
# Dataframe names
df1 = 'Polike_analysis'
df2 = 'alpha_runs'

# Load dataframes
with open(f"/data_extra2/ccortesp/NEXT-100/Xe_cmmssnng/data/{df1}.pkl", "rb") as f1, \
     open(f"/data_extra2/ccortesp/NEXT-100/Xe_cmmssnng/data/{df2}.pkl", "rb") as f2:
    
    data_Po, data_NPo = pickle.load(f1), pickle.load(f2)
    
# Print event statistics for each run
print('Available statistic...')
for i, entry in enumerate(runs_info):
    
    # Run info
    run_id = entry["run_number"]
    
    print(f'Run {run_id}: {data_Po[run_id].shape[0]} events (Polike)')
    print(f'Run {run_id}: {data_NPo[run_id].shape[0]} events (NO-Polike)')

### Active volume and cathode surroundings masks!

In [ ]:
# Time mask
DT_stop = {run["run_number"]: {crudo.cathode_position(run, data_Po, step_back=1)[run["run_number"]], crudo.cathode_position(run, data_NPo, step_back=1)[run["run_number"]],} for run in runs_info}
# Active volume
AV_mask   = {run["run_number"]: {data_Po[run["run_number"]]['DT'] <  data_Po[run["run_number"]], data_NPo[run["run_number"]]['DT'] <  data_NPo[run["run_number"]]} for run in runs_info}
# Cathode surroundings
cath_mask = {run["run_number"]: {data_Po[run["run_number"]]['DT'] >= DT_stop[run["run_number"]], } for run in runs_info}

## Cut Definition
Let's fit this curve to decide the optimal (strong) cut to avoid this Po-like population in the commissioning analysis.

In [ ]:
# Global
n_bins = 75
E1_up  = 1800          # E1 upper limit for the fit
Po_intercept = -56     # Computed using the fit line slope and a desire point: (1800, 250)

for i, entry in enumerate(runs_info):

    # Run info
    run_id   = entry["run_number"]
    run_volt = entry[scan_key] / 1e3

    # Variables
    H1 = sel_data[run_id]['S1h']
    E1 = sel_data[run_id]['S1e']

    # Masking
    H1_mask  = H1[AV_mask[run_id]]
    E1_mask  = E1[AV_mask[run_id]]

    # 2D histogram
    S1h_S1e_map, ex, ey = pt.hist_2D(E1_mask, H1_mask, x_bins=n_bins, y_bins=n_bins)
    S1h_S1e_map         = np.ma.masked_where(S1h_S1e_map==0, S1h_S1e_map)                               # Zero values masked
    
    # Preliminar fitting
    E1_centers, median_S1h = ff.prefit_2D(ex, E1_mask, H1_mask, x_max=E1_up)
    # Linear fitting
    popt, pcov = curve_fit(ff.linear_func, E1_centers, median_S1h)
    print(f'Run {run_id} fit: slope = {popt[0]:.2f}, intercept = {popt[1]:.2f} PE')
    fit_x = np.linspace(E1_centers.min(), E1_centers.max(), 50)
    fit_y = ff.linear_func(fit_x, *popt)
    # Masking line
    print(f'Run {run_id} Po-like masking line: slope = {popt[0]:.2f}, intercept = {Po_intercept} PE')
    mask_y = ff.linear_func(fit_x, popt[0], Po_intercept)

    # Plot
    plt.figure(figsize=(10, 6))
    plt.pcolormesh(ex, ey, S1h_S1e_map.T, cmap=pt.custom_hsv, shading='auto')
    plt.colorbar(label='Counts', shrink=0.75)
    plt.xlabel('Uncorrected S1 Energy (PE)')
    plt.ylabel('S1 Height (PE)')
    plt.title (f'Run {run_id} [{run_volt} kV] (AV)')
    # Fit
    plt.scatter(E1_centers, median_S1h, c='gold', alpha=0.75)
    plt.plot(fit_x, fit_y, c='black', ls='--', lw=1)
    # Masking line
    plt.plot(fit_x, mask_y, c='red', ls='--', lw=1)
    
    plt.gca().set_facecolor("whitesmoke")
    # plt.grid(True)
    # plt.savefig(f'images/polike/Polike_masking_run{run_id}_{scan}.png', bbox_inches='tight')
    plt.show()

### Po-like mask definition

In [ ]:
# Height mask: very important to select this strange population!
Po_mask = {run["run_number"]: sel_data[run["run_number"]]['S1h'] < 0.17 * sel_data[run["run_number"]]['S1e'] - 56 for run in runs_info}
# NOPo_mask = {run["run_number"]: data[run["run_number"]]['S1h'] > 150 for run in runs_info}

# This is the statistic you have
print('After Po masking:')
for i, entry in enumerate(runs_info):
    
    # Run info
    run_id   = entry["run_number"]
    run_volt = entry[scan_key] / 1e3
    
    # Shape of Po-masked data
    shape = sel_data[run_id][Po_mask[run_id]].shape
    print(f'Run {run_id}: {shape[0]} events')

## Base Distributions (with cuts)

How does this population really look like in the detector?

### DT distribution

In [6]:
# # Store important information
# V_drift = {}                      # Drift velocities
# m_DT    = {}                      # Slopes from linear fit

# Global
n_bins = 80
# DT_min = 600                      # Linear fit limits
# DT_max = 0

plt.figure(figsize=(10, 6))

for i, entry in enumerate(runs_info):
    
    # Run info    
    run_id   = entry["run_number"]
    # run_volt = entry[scan_key] / 1e3
    
    # Variables
    DT = sel_data[run_id]['DT']

    # Histogram
    counts, bins, _ = plt.hist(DT, bins=n_bins,
                               color=pt.hist_colors[i % len(pt.hist_colors)], alpha=0.75, 
                               ec=pt.hist_colors[i % len(pt.hist_colors)], lw=1.0, zorder=3)
    
    # Estimated cathode time position + error
    cath_index   = np.argmax(counts)                                                     # Index of the bin with the highest count
    cath_central = (bins[cath_index] + bins[cath_index + 1]) / 2 
    cath_error   = (bins[1] - bins[0]) / 2 
    # Drift velocity + error
    drift_vel       = (pt.N100_hei - pt.EL_gap) / cath_central
    drift_error     = drift_vel * (cath_error / cath_central)
    # V_drift[run_id] = {'CV': drift_vel, 'Error': drift_error}
    
    print(f'Run {run_id}:')
    print(f'Cathode time position = ({cath_central:.2f} +/- {cath_error:.2f}) μs')
    print(f'Drift velocity = ({drift_vel:.2f} +/- {drift_error:.2f}) mm/μs \n') 
    
    plt.axvline(x=cath_central, c='red', ls='--', lw=0.75)                               # Cathode line
    
    # # Linear fit
    # DT_max = DT_stop[run_id]
    # DT_centers, DT_fit = ff.prefit_1D(bins, DT, counts, DT_min, DT_max)
    # popt, pcov = curve_fit(ff.linear_func, DT_centers, DT_fit)
    # # m_DT[run_id] = {'CV': popt[0], 'Err': pcov[0, 0]}
    
    # # Plot fit
    # fit_x = np.linspace(DT_centers.min(), DT_centers.max(), n_bins)
    # fit_y = ff.linear_func(fit_x, *popt)
    # plt.plot(fit_x, fit_y, c='black', ls='--', lw=0.75)

# Plot
plt.xlabel('$\Delta$T ($\mu$s)')
plt.ylabel('Counts')
plt.yscale('log')
plt.title(f'$\Delta$T Distribution')

plt.gca().set_facecolor("whitesmoke")
plt.grid(True, which='both', ls='--', lw=0.75, alpha=0.50)
plt.show()

### Radial distribution

In [7]:
# Global
n_bins = 125

plt.figure(figsize=(10, 6))

for i, entry in enumerate(runs_info):
    
    # Run info
    run_id   = entry["run_number"]
    # run_volt = entry[scan_key] / 1e3

    # Variables
    R2 = sel_data[run_id]['X']**2 + sel_data[run_id]['Y']**2
    
    # Masking
    R2_av_mask   = R2[AV_mask[run_id]]
    R2_cath_mask = R2[cath_mask[run_id]]
    
    # Histograms
    counts_av, bins_av, _     = plt.hist(R2_av_mask, bins=n_bins, histtype='step', lw=1.5, range=(0, 1.25*pt.N100_rad**2),
                                         ec=pt.hist_colors[(i + 1) % len(pt.hist_colors)], label=f'Active Volume')
    counts_cath, bins_cath, _ = plt.hist(R2_cath_mask, bins=n_bins, histtype='step', lw=1.5, range=(0, 1.25*pt.N100_rad**2),
                                         ec=pt.hist_colors[(i + 5) % len(pt.hist_colors)], label=f'Cathode')
        
# Plot
plt.xlabel('R$^{2}$ (mm$^{2}$)')
plt.ylabel('Counts')
plt.yscale('log')
plt.title(f'Radial Distribution: Run {run_id}')
plt.ticklabel_format(axis='x', style='sci', scilimits=(3, 3))
plt.legend(loc='upper right')
# Radial limit
plt.axvline(x=pt.N100_rad**2, c='gray', ls='--', lw=1)
plt.text(pt.N100_rad**2, counts_av[0], 'detector limit', c='gray', va='center',  ha='right',  rotation=90)

plt.gca().set_facecolor("whitesmoke")
plt.show()

#### Active volume

In [8]:
# Global
n_bins = 25

for i, entry in enumerate(runs_info):
    
    # Run info    
    run_id   = entry["run_number"]
    # run_volt = entry[scan_key] / 1e3
    # run_long = entry["duration"]
    
    # Variables    
    X = sel_data[run_id]['X']
    Y = sel_data[run_id]['Y']
    
    # Masking
    X_mask = X[AV_mask[run_id]]
    Y_mask = Y[AV_mask[run_id]]
    
    # Map
    xy_map, ex, ey = pt.mapping(X_mask, Y_mask, xy_bins=n_bins, pos=True, norm=True)
    xy_map         = np.ma.masked_where(xy_map==0, xy_map)                               # Zero values masked
    # xy_map         = np.clip(xy_map, 0.75, 1.25)
    
    # Plot
    plt.figure(figsize=(8, 8))
    plt.pcolormesh(ex, ey, xy_map.T, cmap=pt.custom_hsv, shading='auto')
    plt.colorbar(label='A. U.', shrink=0.75)
    plt.xlabel('X (mm)')
    plt.ylabel('Y (mm)')
    plt.title (f'Position Map: Run {run_id} (AV)') 
    
    plt.gca().add_patch(pt.plot_circle(pt.N100_rad))
    plt.gca().set_aspect('equal', adjustable='box')
    plt.gca().set_facecolor("whitesmoke")
    plt.show()

#### Cathode surface

In [9]:
# Global
n_bins = 25

for i, entry in enumerate(runs_info):
    
    # Run info    
    run_id   = entry["run_number"]
    # run_volt = entry[scan_key] / 1e3
    # run_long = entry["duration"]
    
    # Variables    
    X = sel_data[run_id]['X']
    Y = sel_data[run_id]['Y']
    
    # Masking
    X_mask = X[cath_mask[run_id]]
    Y_mask = Y[cath_mask[run_id]]
    
    # Map
    xy_map, ex, ey = pt.mapping(X_mask, Y_mask, xy_bins=n_bins, pos=True, norm=True)
    xy_map         = np.ma.masked_where(xy_map==0, xy_map)                               # Zero values masked
    # xy_map         = np.clip(xy_map, 0.75, 1.25)
    
    # Plot
    plt.figure(figsize=(8, 8))
    plt.pcolormesh(ex, ey, xy_map.T, cmap=pt.custom_hsv, shading='auto')
    plt.colorbar(label='A. U.', shrink=0.75)
    plt.xlabel('X (mm)')
    plt.ylabel('Y (mm)')
    plt.title (f'Position Map: Run {run_id} (Cathode)') 
    
    plt.gca().add_patch(pt.plot_circle(pt.N100_rad))
    plt.gca().set_aspect('equal', adjustable='box')
    plt.gca().set_facecolor("whitesmoke")
    plt.show()

### $S1$ Energy

In [10]:
# Global
n_bins = 100

plt.figure(figsize=(10, 6))

for i, entry in enumerate(runs_info):
    
    # Run info
    run_id   = entry["run_number"]
    # run_volt = entry[scan_key] / 1e3

    # Variables
    E1 = sel_data[run_id]['S1e']
    
    # Masking
    E1_av_mask   = E1[AV_mask[run_id]]
    E1_cath_mask = E1[cath_mask[run_id]]
    
    # Histograms
    counts_av, bins_av, _     = plt.hist(E1_av_mask, bins=n_bins, histtype='step', lw=1.5,
                                         ec=pt.hist_colors[(i + 1) % len(pt.hist_colors)], label=f'Active Volume')
    counts_cath, bins_cath, _ = plt.hist(E1_cath_mask, bins=n_bins, histtype='step', lw=1.5,
                                         ec=pt.hist_colors[(i + 5) % len(pt.hist_colors)], label=f'Cathode')
        
# Plot
plt.xlabel('Energy (PE)')
plt.ylabel('Counts')
# plt.yscale('log')
plt.title(f'S1 Energy Distribution: Run {run_id}')
plt.legend(loc='upper right')

plt.gca().set_facecolor("whitesmoke")
plt.show()

#### Active volume

In [11]:
# Global
n_bins = 25

for i, entry in enumerate(runs_info):
    
    # Run info    
    run_id   = entry["run_number"]
    # run_volt = entry[scan_key] / 1e3
    # run_long = entry["duration"]
    
    # Variables    
    X  = sel_data[run_id]['X']
    Y  = sel_data[run_id]['Y']
    E1 = sel_data[run_id]['S1e']
    
    # Masking
    X_mask  = X[AV_mask[run_id]]
    Y_mask  = Y[AV_mask[run_id]]
    E1_mask = E1[AV_mask[run_id]] 
    
    # Map
    S1e_map, ex, ey = pt.mapping(X_mask, Y_mask, wei=E1_mask, xy_bins=n_bins, norm=True)
    S1e_map         = np.ma.masked_where(S1e_map==0, S1e_map)                               # Zero values masked
    S1e_map         = np.clip(S1e_map, 0.75, 1.15)
    
    # Plot
    plt.figure(figsize=(8, 8))
    plt.pcolormesh(ex, ey, S1e_map.T, cmap=pt.custom_hsv, shading='auto')
    plt.colorbar(label='A. U.', shrink=0.75)
    plt.xlabel('X (mm)')
    plt.ylabel('Y (mm)')
    plt.title (f'Uncorrected S1 Energy Map: Run {run_id} (AV)') 
    
    plt.gca().add_patch(pt.plot_circle(pt.N100_rad))
    plt.gca().set_aspect('equal', adjustable='box')
    plt.gca().set_facecolor("whitesmoke")
    plt.show()

#### Cathode surface

In [12]:
# Global
n_bins = 25

for i, entry in enumerate(runs_info):
    
    # Run info    
    run_id   = entry["run_number"]
    # run_volt = entry[scan_key] / 1e3
    # run_long = entry["duration"]
    
    # Variables    
    X  = sel_data[run_id]['X']
    Y  = sel_data[run_id]['Y']
    E1 = sel_data[run_id]['S1e']
    
    # Masking
    X_mask  = X[cath_mask[run_id]]
    Y_mask  = Y[cath_mask[run_id]]
    E1_mask = E1[cath_mask[run_id]] 
    
    # Map
    S1e_map, ex, ey = pt.mapping(X_mask, Y_mask, wei=E1_mask, xy_bins=n_bins, norm=True)
    S1e_map         = np.ma.masked_where(S1e_map==0, S1e_map)                               # Zero values masked
    # S1e_map         = np.clip(S1e_map, 0.75, 1.25)
    
    # Plot
    plt.figure(figsize=(8, 8))
    plt.pcolormesh(ex, ey, S1e_map.T, cmap=pt.custom_hsv, shading='auto')
    plt.colorbar(label='A. U.', shrink=0.75)
    plt.xlabel('X (mm)')
    plt.ylabel('Y (mm)')
    plt.title (f'Uncorrected S1 Energy Map: Run {run_id} (Cathode)') 
    
    plt.gca().add_patch(pt.plot_circle(pt.N100_rad))
    plt.gca().set_aspect('equal', adjustable='box')
    plt.gca().set_facecolor("whitesmoke")
    plt.show()

### $S2$ Energy

In [13]:
# Global
n_bins = 100

plt.figure(figsize=(10, 6))

for i, entry in enumerate(runs_info):
    
    # Run info
    run_id   = entry["run_number"]
    # run_volt = entry[scan_key] / 1e3

    # Variables
    E2 = sel_data[run_id]['S2e']
    
    # Masking
    E2_av_mask   = E2[AV_mask[run_id]]
    E2_cath_mask = E2[cath_mask[run_id]]
    
    # Histograms
    counts_av, bins_av, _     = plt.hist(E2_av_mask, bins=n_bins, histtype='step', lw=1.5,
                                         ec=pt.hist_colors[(i + 1) % len(pt.hist_colors)], label=f'Active Volume')
    counts_cath, bins_cath, _ = plt.hist(E2_cath_mask, bins=n_bins, histtype='step', lw=1.5,
                                         ec=pt.hist_colors[(i + 5) % len(pt.hist_colors)], label=f'Cathode')
        
# Plot
plt.xlabel('Energy (PE)')
plt.ylabel('Counts')
# plt.yscale('log')
plt.ticklabel_format(axis='x', style='sci', scilimits=(3, 3))
plt.title(f'S2 Energy Distribution: Run {run_id}')
plt.legend(loc='upper right')

plt.gca().set_facecolor("whitesmoke")
plt.show()

#### Active volume

In [14]:
# Global
n_bins = 25

for i, entry in enumerate(runs_info):
    
    # Run info    
    run_id   = entry["run_number"]
    # run_volt = entry[scan_key] / 1e3
    # run_long = entry["duration"]
    
    # Variables    
    X  = sel_data[run_id]['X']
    Y  = sel_data[run_id]['Y']
    E2 = sel_data[run_id]['S2e']
    
    # Masking
    X_mask  = X[AV_mask[run_id]]
    Y_mask  = Y[AV_mask[run_id]]
    E2_mask = E2[AV_mask[run_id]] 
    
    # Map
    S2e_map, ex, ey = pt.mapping(X_mask, Y_mask, wei=E2_mask, xy_bins=n_bins, norm=True)
    S2e_map         = np.ma.masked_where(S2e_map==0, S2e_map)                               # Zero values masked
    # S2e_map         = np.clip(S2e_map, 0.75, 1.25)
    
    # Plot
    plt.figure(figsize=(8, 8))
    plt.pcolormesh(ex, ey, S2e_map.T, cmap=pt.custom_hsv, shading='auto')
    plt.colorbar(label='A. U.', shrink=0.75)
    plt.xlabel('X (mm)')
    plt.ylabel('Y (mm)')
    plt.title (f'Uncorrected S2 Energy Map: Run {run_id} (AV)') 
    
    plt.gca().add_patch(pt.plot_circle(pt.N100_rad))
    plt.gca().set_aspect('equal', adjustable='box')
    plt.gca().set_facecolor("whitesmoke")
    plt.show()

#### Cathode surface

In [15]:
# Global
n_bins = 25

for i, entry in enumerate(runs_info):
    
    # Run info    
    run_id   = entry["run_number"]
    # run_volt = entry[scan_key] / 1e3
    # run_long = entry["duration"]
    
    # Variables    
    X  = sel_data[run_id]['X']
    Y  = sel_data[run_id]['Y']
    E2 = sel_data[run_id]['S2e']
    
    # Masking
    X_mask  = X[cath_mask[run_id]]
    Y_mask  = Y[cath_mask[run_id]]
    E2_mask = E2[cath_mask[run_id]] 
    
    # Map
    S2e_map, ex, ey = pt.mapping(X_mask, Y_mask, wei=E2_mask, xy_bins=n_bins, norm=True)
    S2e_map         = np.ma.masked_where(S2e_map==0, S2e_map)                               # Zero values masked
    S2e_map         = np.clip(S2e_map, 0.75, 1.25)
    
    # Plot
    plt.figure(figsize=(8, 8))
    plt.pcolormesh(ex, ey, S2e_map.T, cmap=pt.custom_hsv, shading='auto')
    plt.colorbar(label='A. U.', shrink=0.75)
    plt.xlabel('X (mm)')
    plt.ylabel('Y (mm)')
    plt.title (f'Uncorrected S2 Energy Map: Run {run_id} (Cathode)') 
    
    plt.gca().add_patch(pt.plot_circle(pt.N100_rad))
    plt.gca().set_aspect('equal', adjustable='box')
    plt.gca().set_facecolor("whitesmoke")
    plt.show()

### $S1$ Width

In [16]:
# Global
n_bins = 100

plt.figure(figsize=(10, 6))

for i, entry in enumerate(runs_info):
    
    # Run info
    run_id   = entry["run_number"]
    # run_volt = entry[scan_key] / 1e3

    # Variables
    W1 = sel_data[run_id]['S1w'] / 1e3
    
    # Masking
    W1_av_mask   = W1[AV_mask[run_id]]
    W1_cath_mask = W1[cath_mask[run_id]]
    
    # Histograms
    counts_av, bins_av, _     = plt.hist(W1_av_mask, bins=n_bins, histtype='step', lw=1.5,
                                         ec=pt.hist_colors[(i + 1) % len(pt.hist_colors)], label=f'Active Volume')
    counts_cath, bins_cath, _ = plt.hist(W1_cath_mask, bins=n_bins, histtype='step', lw=1.5,
                                         ec=pt.hist_colors[(i + 5) % len(pt.hist_colors)], alpha=0.75, label=f'Cathode')
        
# Plot
plt.xlabel('Width ($\mu$s)')
plt.ylabel('Counts')
plt.yscale('log')
plt.title(f'S1 Width Distribution: Run {run_id}')
plt.legend(loc='upper right')

plt.gca().set_facecolor("whitesmoke")
plt.show()

#### Active volume

In [17]:
# Global
n_bins = 25

for i, entry in enumerate(runs_info):
    
    # Run info    
    run_id   = entry["run_number"]
    # run_volt = entry[scan_key] / 1e3
    # run_long = entry["duration"]
    
    # Variables    
    X  = sel_data[run_id]['X']
    Y  = sel_data[run_id]['Y']
    W1 = sel_data[run_id]['S1w'] / 1e3
    
    # Masking
    X_mask  = X[AV_mask[run_id]]
    Y_mask  = Y[AV_mask[run_id]]
    W1_mask = W1[AV_mask[run_id]] 
    
    # Map
    S1w_map, ex, ey = pt.mapping(X_mask, Y_mask, wei=W1_mask, xy_bins=n_bins)
    S1w_map         = np.ma.masked_where(S1w_map==0, S1w_map)                               # Zero values masked
    S1w_map         = np.clip(S1w_map, 0, 1.25)
    
    # Plot
    plt.figure(figsize=(8, 8))
    plt.pcolormesh(ex, ey, S1w_map.T, cmap=pt.custom_hsv, shading='auto')
    plt.colorbar(label='Width ($\mu$s)', shrink=0.75)
    plt.xlabel('X (mm)')
    plt.ylabel('Y (mm)')
    plt.title (f'S1 Width Map: Run {run_id} (AV)') 
    
    plt.gca().add_patch(pt.plot_circle(pt.N100_rad))
    plt.gca().set_aspect('equal', adjustable='box')
    plt.gca().set_facecolor("whitesmoke")
    plt.show()

#### Cathode surface

In [18]:
# Global
n_bins = 25

for i, entry in enumerate(runs_info):
    
    # Run info    
    run_id   = entry["run_number"]
    # run_volt = entry[scan_key] / 1e3
    # run_long = entry["duration"]
    
    # Variables    
    X  = sel_data[run_id]['X']
    Y  = sel_data[run_id]['Y']
    W1 = sel_data[run_id]['S1w'] / 1e3
    
    # Masking
    X_mask  = X[cath_mask[run_id]]
    Y_mask  = Y[cath_mask[run_id]]
    W1_mask = W1[cath_mask[run_id]] 
    
    # Map
    S1w_map, ex, ey = pt.mapping(X_mask, Y_mask, wei=W1_mask, xy_bins=n_bins)
    S1w_map         = np.ma.masked_where(S1w_map==0, S1w_map)                               # Zero values masked
    S1w_map         = np.clip(S1w_map, 0, 1.25)
    
    # Plot
    plt.figure(figsize=(8, 8))
    plt.pcolormesh(ex, ey, S1w_map.T, cmap=pt.custom_hsv, shading='auto')
    plt.colorbar(label='Width ($\mu$s)', shrink=0.75)
    plt.xlabel('X (mm)')
    plt.ylabel('Y (mm)')
    plt.title (f'S1 Width Map: Run {run_id} (Cathode)') 
    
    plt.gca().add_patch(pt.plot_circle(pt.N100_rad))
    plt.gca().set_aspect('equal', adjustable='box')
    plt.gca().set_facecolor("whitesmoke")
    plt.show()

### $S2$ Width

In [19]:
# Global
n_bins = 125

plt.figure(figsize=(10, 6))

for i, entry in enumerate(runs_info):
    
    # Run info
    run_id   = entry["run_number"]
    # run_volt = entry[scan_key] / 1e3

    # Variables
    W2 = sel_data[run_id]['S2w']
    
    # Masking
    W2_av_mask   = W2[AV_mask[run_id]]
    W2_cath_mask = W2[cath_mask[run_id]]
    
    # Histograms
    counts_av, bins_av, _     = plt.hist(W2_av_mask, bins=n_bins, histtype='step', lw=1.5,
                                         ec=pt.hist_colors[(i + 1) % len(pt.hist_colors)], label=f'Active Volume')
    counts_cath, bins_cath, _ = plt.hist(W2_cath_mask, bins=n_bins, histtype='step', lw=1.5,
                                         ec=pt.hist_colors[(i + 5) % len(pt.hist_colors)], alpha=0.75, label=f'Cathode')
        
# Plot
plt.xlabel('Width ($\mu$s)')
# plt.xlim(0, 250)
plt.ylabel('Counts')
# plt.yscale('log')
plt.title(f'S2 Width Distribution: Run {run_id}')
plt.legend(loc='upper right')

plt.gca().set_facecolor("whitesmoke")
plt.show()

#### Active volume

In [20]:
# Global
n_bins = 25

for i, entry in enumerate(runs_info):
    
    # Run info    
    run_id   = entry["run_number"]
    # run_volt = entry[scan_key] / 1e3
    # run_long = entry["duration"]
    
    # Variables    
    X  = sel_data[run_id]['X']
    Y  = sel_data[run_id]['Y']
    W2 = sel_data[run_id]['S2w']
    
    # Masking
    X_mask  = X[AV_mask[run_id]]
    Y_mask  = Y[AV_mask[run_id]]
    W2_mask = W2[AV_mask[run_id]] 
    
    # Map
    S2w_map, ex, ey = pt.mapping(X_mask, Y_mask, wei=W2_mask, xy_bins=n_bins)
    S2w_map         = np.ma.masked_where(S2w_map==0, S2w_map)                               # Zero values masked
    S2w_map         = np.clip(S2w_map, 0, 160)
    
    # Plot
    plt.figure(figsize=(8, 8))
    plt.pcolormesh(ex, ey, S2w_map.T, cmap=pt.custom_hsv, shading='auto')
    plt.colorbar(label='Width ($\mu$s)', shrink=0.75)
    plt.xlabel('X (mm)')
    plt.ylabel('Y (mm)')
    plt.title (f'S2 Width Map: Run {run_id} (AV)') 
    
    plt.gca().add_patch(pt.plot_circle(pt.N100_rad))
    plt.gca().set_aspect('equal', adjustable='box')
    plt.gca().set_facecolor("whitesmoke")
    plt.show()

#### Cathode surface

In [21]:
# Global
n_bins = 25

for i, entry in enumerate(runs_info):
    
    # Run info    
    run_id   = entry["run_number"]
    # run_volt = entry[scan_key] / 1e3
    # run_long = entry["duration"]
    
    # Variables    
    X  = sel_data[run_id]['X']
    Y  = sel_data[run_id]['Y']
    W2 = sel_data[run_id]['S2w']
    
    # Masking
    X_mask  = X[cath_mask[run_id]]
    Y_mask  = Y[cath_mask[run_id]]
    W2_mask = W2[cath_mask[run_id]] 
    
    # Map
    S2w_map, ex, ey = pt.mapping(X_mask, Y_mask, wei=W2_mask, xy_bins=n_bins)
    S2w_map         = np.ma.masked_where(S2w_map==0, S2w_map)                               # Zero values masked
    S2w_map         = np.clip(S2w_map, 0, 130)
    
    # Plot
    plt.figure(figsize=(8, 8))
    plt.pcolormesh(ex, ey, S2w_map.T, cmap=pt.custom_hsv, shading='auto')
    plt.colorbar(label='Width ($\mu$s)', shrink=0.75)
    plt.xlabel('X (mm)')
    plt.ylabel('Y (mm)')
    plt.title (f'S2 Width Map: Run {run_id} (Cathode)') 
    
    plt.gca().add_patch(pt.plot_circle(pt.N100_rad))
    plt.gca().set_aspect('equal', adjustable='box')
    plt.gca().set_facecolor("whitesmoke")
    plt.show()

## Commissioning Plots

### $S1e$ vs $S2e$

In [36]:
# Global
rad_lim = 200
n_bins = 75

for i, entry in enumerate(runs_info):
    
    # Run info
    run_id   = entry["run_number"]
    # run_volt = entry[scan_key] / 1e3
    run_long = entry["duration"]
    run_DAQe = (entry["OK"] / (entry["OK"] + entry["LOST"]))

    # Variables    
    E1 = sel_data[run_id]['S1e']
    E2 = sel_data[run_id]['S2e']
    R  = sel_data[run_id]['R']
    
    # Masking
    rad_mask = (R <= rad_lim)
    E1_mask  = E1[AV_mask[run_id] & rad_mask]
    E2_mask  = E2[AV_mask[run_id] & rad_mask]
    
    # 2D histogram: normalized by corrected DAQ data taking time
    S1e_S2e_map, ex, ey = pt.hist_2D(E2_mask, E1_mask, x_bins=n_bins, y_bins=n_bins)
    S1e_S2e_map         = S1e_S2e_map / (run_long * run_DAQe)                           
    S1e_S2e_map         = np.ma.masked_where(S1e_S2e_map==0, S1e_S2e_map)                # Zero values masked
    # S1e_S2e_map = np.clip(S1e_S2e_map, 0, 50)                                           # Clipped values in colorbar
        
    # Plot
    plt.figure(figsize=(10, 6))
    plt.pcolormesh(ex, ey, S1e_S2e_map.T, cmap=pt.custom_hsv, shading='auto')    
    cbar = plt.colorbar(label='Rate (Hz)', shrink=0.75)
    cbar.formatter.set_powerlimits((-2, -2))
    plt.xlabel('Uncorrected S2 Energy (PE)')
    plt.ylabel('Uncorrected S1 Energy (PE)')
    plt.ticklabel_format(axis='x', style='sci', scilimits=(3, 3))
    plt.title(f'Run {run_id}: AV & (R <= {rad_lim} mm)')

    plt.gca().set_facecolor("whitesmoke")
    plt.show()

### $S1e$ vs DT

In [39]:
# Global
rad_lim = 200
n_bins = 75

for i, entry in enumerate(runs_info):
    
    # Run info
    run_id   = entry["run_number"]
    # run_volt = entry[scan_key] / 1e3
    run_long = entry["duration"]
    run_DAQe = (entry["OK"] / (entry["OK"] + entry["LOST"]))

    # Variables    
    E1 = sel_data[run_id]['S1e']
    DT = sel_data[run_id]['DT']
    R  = sel_data[run_id]['R']
    
    # Masking
    rad_mask = (R <= rad_lim)
    E1_mask  = E1[AV_mask[run_id] & rad_mask]
    DT_mask  = DT[AV_mask[run_id] & rad_mask]
    
    # 2D histogram: normalized by corrected DAQ data taking time
    S1e_DT_map, ex, ey = pt.hist_2D(DT, E1, x_bins=n_bins, y_bins=n_bins)
    S1e_DT_map         = S1e_DT_map / (run_long * run_DAQe)                           
    S1e_DT_map         = np.ma.masked_where(S1e_DT_map==0, S1e_DT_map)                # Zero values masked
    # S1e_DT_map = np.clip(S1e_DT_map, 0, 50)                                           # Clipped values in colorbar
        
    # Plot
    plt.figure(figsize=(10, 6))
    plt.pcolormesh(ex, ey, S1e_DT_map.T, cmap=pt.custom_hsv, shading='auto')    
    cbar = plt.colorbar(label='Rate (Hz)', shrink=0.75)
    cbar.formatter.set_powerlimits((-2, -2))
    plt.xlabel('Uncorrected S2 Energy (PE)')
    plt.ylabel('Uncorrected S1 Energy (PE)')
    plt.ticklabel_format(axis='x', style='sci', scilimits=(3, 3))
    plt.title(f'Run {run_id}: AV & (R <= {rad_lim} mm)')

    plt.gca().set_facecolor("whitesmoke")
    plt.show()

### $S2e$ vs DT

In [42]:
# Global
rad_lim = 200
n_bins = 75

for i, entry in enumerate(runs_info):
    
    # Run info
    run_id   = entry["run_number"]
    # run_volt = entry[scan_key] / 1e3
    run_long = entry["duration"]
    run_DAQe = (entry["OK"] / (entry["OK"] + entry["LOST"]))

    # Variables    
    E2 = sel_data[run_id]['S2e']
    DT = sel_data[run_id]['DT']
    R  = sel_data[run_id]['R']
    
    # Masking
    rad_mask = (R <= rad_lim)
    E2_mask  = E2[AV_mask[run_id] & rad_mask]
    DT_mask  = DT[AV_mask[run_id] & rad_mask]
    
    # 2D histogram: normalized by corrected DAQ data taking time
    S2e_DT_map, ex, ey = pt.hist_2D(DT, E2, x_bins=n_bins, y_bins=n_bins)
    S2e_DT_map         = S2e_DT_map / (run_long * run_DAQe)                           
    S2e_DT_map         = np.ma.masked_where(S2e_DT_map==0, S2e_DT_map)                # Zero values masked
    # S2e_DT_map = np.clip(S2e_DT_map, 0, 50)                                           # Clipped values in colorbar
        
    # Plot
    plt.figure(figsize=(10, 6))
    plt.pcolormesh(ex, ey, S2e_DT_map.T, cmap=pt.custom_hsv, shading='auto')    
    cbar = plt.colorbar(label='Rate (Hz)', shrink=0.75)
    cbar.formatter.set_powerlimits((-2, -2))
    plt.xlabel('Uncorrected S2 Energy (PE)')
    plt.ylabel('Uncorrected S1 Energy (PE)')
    plt.ticklabel_format(axis='x', style='sci', scilimits=(3, 3))
    plt.title(f'Run {run_id}: AV & (R <= {rad_lim} mm)')

    plt.gca().set_facecolor("whitesmoke")
    plt.show()